# Model Analysis

In [ ]:
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec
from packages.TPPMI.ppmi_model import PPMIModel
from packages.TPPMI.tppmi_model import TPPMIModel

from embedding_visualization import plot_temporal_changing_embedding, \
    plot_cosine_similarity, plot_word_vectors_tppmi, plot_cosine_similarity_tppmi

In [ ]:
model_path = Path("model/")
ppmi_size = "medium"
ppmi_path = Path(f"data/ppmi-matrices/{ppmi_size}")

In [ ]:
model_static = Word2Vec.load(str(model_path / "word2vec.model"))

In [ ]:
def print_most_similar_cade(models, target_word, top_n=3):
    print(f"Word: {target_word}")
    for key, value in models.items():
        print(f"Month: {key.split('_')[1].capitalize()}")
        try:
            print(value.wv.most_similar(target_word, topn=top_n))
        except KeyError:
            print(f"{target_word} not in vocab")
        print("--------------------------------")

## Load models for months

### Cade

In [ ]:
model_path_monthly = model_path / "monthly"
model_filenames = glob(str(model_path_monthly / "*.model"))

In [ ]:
models_monthly = {f"model_{model_file.split('_')[1][0:3].lower()}":Word2Vec.load(model_file) for model_file in tqdm(model_filenames)}

In [ ]:
models_monthly.keys()

Order the models

In [ ]:
order = ['jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr']

# Sort the keys based on the desired month order
sorted_keys = sorted(models_monthly.keys(), key=lambda x: order.index(x.split('_')[1]))

# Create a sorted dictionary using a dictionary comprehension
models_monthly = {key: models_monthly[key] for key in sorted_keys}

### TPPMI

In [ ]:
ppmi_filenames = glob(str(ppmi_path / "*.txt"))
ppmi_dfs = {filename.split("2022-")[1][0:2]: pd.read_csv(filename, sep=" ") for filename in tqdm(ppmi_filenames)}

Set the vocab to be the index (instead of being the first attribute)

In [ ]:
ppmi_dfs = {key:ppmi_df.set_index(ppmi_df.columns[0]) for key, ppmi_df in ppmi_dfs.items()}

Create ppmi_model objects

In [ ]:
ppmi_models = {key: PPMIModel.construct_from_data(ppmi_df) for key, ppmi_df in ppmi_dfs.items()}

In [ ]:
tppmi_model = TPPMIModel(ppmi_models)

In [ ]:
print(f"Size of the vocabulary: {tppmi_model.get_vocabulary_size()}")

## Release of Brittney Griner

### Overview

On December 8, 2022, Russia and the United States conducted a prisoner exchange, trading Brittney Griner, an American basketball player, for Viktor Bout, a Russian arms dealer. Griner, a WNBA champion star and Team USA Olympic athlete, had been convicted of smuggling and possession of cannabis in Russia earlier in 2022 and sentenced to nine years in prison.

Source: [Wikipedia article](https://en.wikipedia.org/wiki/Viktor_Bout%E2%80%93Brittney_Griner_prisoner_exchange)

In [ ]:
target_word_griner = "brittney"

In [ ]:
selected_months_griner = ["sep", "oct", "nov", "dec", "jan"]
selected_models_monthly_griner = {f"model_{month}": models_monthly[f"model_{month}"] for month in selected_months_griner}

In [ ]:
title_griner = f"Evolution of the word {target_word_griner} over time"
subtitle_griner = f"Period: {' - '.join([month.capitalize() for month in selected_months_griner])}"

In [ ]:
print_most_similar_cade(selected_models_monthly_griner, target_word_griner, top_n=3)

#### CADE

### Temporally changing embeddings

In [ ]:
plot_temporal_changing_embedding(target_word_griner, selected_models_monthly_griner, top_n=3, title=title_griner, subtitle=subtitle_griner, use_tsne=False)

### Cosine similarities

In [ ]:
test_words_griner = ["russia", "karen", "putin"]

In [ ]:
plot_cosine_similarity(target_word_griner, test_words_griner, selected_models_monthly_griner, event="dec", event_name="Brittney Griner is released from prison")

### TPPMI

In [ ]:
target_word_griner = "brittney"
selected_months = [9, 10, 11, 12]
tppmi_griner = tppmi_model.get_tppmi(test_words_griner + [target_word_griner], selected_months=selected_months)

In [ ]:
test_words_griner = ["putin", "victor"]
plot_word_vectors_tppmi(tppmi_model.get_2d_representation([target_word_griner] + test_words_griner, selected_months=selected_months, use_tsne=False))

In [ ]:
#plot_cosine_similarity_tppmi(target_word_griner, test_words_griner, tppmi_model, selected_months)

## Elon Musk Twitter takeover

### Overview

Business magnate Elon Musk initiated an acquisition of American social media company Twitter, Inc. on April 14, 2022, and concluded it on October 27, 2022. Musk had begun buying shares of the company in January 2022, becoming its largest shareholder by April with a 9.1 percent ownership stake

source: [Wikipedia article](https://en.wikipedia.org/wiki/Acquisition_of_Twitter_by_Elon_Musk#:~:text=Business%20magnate%20Elon%20Musk%20initiated,a%209.1%20percent%20ownership%20stake.)

In [ ]:
target_word_twitter = "twitter"

In [ ]:
selected_months_twitter = ["sep", "oct", "nov", "dec"]
selected_models_monthly_twitter = {f"model_{month}": models_monthly[f"model_{month}"] for month in selected_months_twitter}

In [ ]:
title_twitter = f"Evolution of the word {target_word_twitter} over time"
subtitle_twitter = f"Period: {' - '.join([month.capitalize() for month in selected_months_twitter])}"

In [ ]:
print_most_similar_cade(selected_models_monthly_twitter, target_word_twitter)

### Temporaly changing embeddings

In [ ]:
plot_temporal_changing_embedding(target_word_twitter, selected_models_monthly_twitter, top_n=2, title=title_twitter, subtitle=subtitle_twitter, use_tsne=True)

### Cosine similarities

In [ ]:
test_words_twitter = ["facebook",  "elon", "blue", "liberal"]

In [ ]:
plot_cosine_similarity(target_word_twitter, test_words_twitter, models_monthly, "nov", "Elon Musk takes over Twitter")

### TPPMI

In [ ]:
tppmi_twitter = tppmi_model.get_tppmi(test_words_twitter + [target_word_twitter])

In [ ]:
selected_months = [9, 10, 11, 12]
plot_word_vectors_tppmi(tppmi_model.get_2d_representation(test_words_twitter + [target_word_twitter], selected_months=selected_months, use_tsne=False))

In [ ]:
plot_cosine_similarity_tppmi("twitter", ["facebook", "elon", "musk"], tppmi_model, selected_months = [10, 11, 12])

## Attack on Paul Pelosi

### Overview

On October 28, 2022, an intruder attacked Paul Pelosi, the 82-year-old husband of Nancy Pelosi, then the Speaker of the United States House of Representatives. The assailant beat Paul Pelosi with a hammer during a home invasion burglary of the couple's residence in Pacific Heights, San Francisco. He was seriously injured and underwent surgery for his fractured skull.

source: [Wikipedia article](https://en.wikipedia.org/wiki/Attack_on_Paul_Pelosi)

In [ ]:
target_word_pelosi = "paul"

In [ ]:
selected_months_pelosi = ["oct", "nov", "dec", "jan"]
selected_models_monthly_pelosi = {f"model_{month}": models_monthly[f"model_{month}"] for month in selected_months_pelosi}

In [ ]:
title_pelosi = f"Evolution of the word {target_word_pelosi} over time"
subtitle_pelosi = f"Period: {' - '.join([month.capitalize() for month in selected_months_pelosi])}"

In [ ]:
print_most_similar_cade(selected_models_monthly_pelosi, target_word_pelosi, top_n = 4)

### Temporally changing embeddings

In [ ]:
plot_temporal_changing_embedding(target_word_pelosi, selected_models_monthly_pelosi, top_n=3, title=title_pelosi, subtitle=subtitle_pelosi, use_tsne=True)

### Cosine similarities

In [ ]:
print_most_similar_cade(models_monthly, target_word_pelosi)

In [ ]:
test_words_pelosi = ["attack", "democrats", "invasion", "hammer", "intruder"]

In [ ]:
plot_cosine_similarity(target_word_pelosi, test_words_pelosi, models_monthly, event="nov", event_name="Attack on Paul Pelosi")

### TPPMI

In [ ]:
tppmi_pelosi = tppmi_model.get_tppmi(test_words_pelosi + [target_word_pelosi])

In [ ]:
test_words_pelosi = ["attack", "democrats", "invasion"]
selected_months_pelosi = [10, 11, 12]
plot_word_vectors_tppmi(tppmi_model.get_2d_representation(test_words_pelosi + [target_word_pelosi], use_tsne=False, selected_months = selected_months_pelosi))

In [ ]:
plot_cosine_similarity_tppmi(target_word_pelosi, test_words_pelosi, tppmi_model, selected_months = selected_months_pelosi)

## Colorado Springs shooting at LGBTQ nightclub

### Overview

On November 19–20, 2022, an anti-LGBT-motivated mass shooting occurred at Club Q, a gay bar in Colorado Springs, Colorado, United States. Five people were murdered, and 25 others were injured, 19 of them by gunfire. The shooter, 22-year-old Anderson Lee Aldrich, was also injured while being restrained, and was taken to a local hospital

source: [wikipedia article](https://en.wikipedia.org/wiki/Colorado_Springs_nightclub_shooting)

In [ ]:
target_word_colorado_springs = "nightclub"

In [ ]:
selected_months_colorado_springs = ["oct", "nov", "dec", "jan"]
selected_models_monthly_colorado_springs = {f"model_{month}": models_monthly[f"model_{month}"] for month in selected_months_colorado_springs}

In [ ]:
title_colorado_springs = f"Evolution of the word {target_word_colorado_springs} over time"
subtitle_colorado_springs = f"Period: {' - '.join([month.capitalize() for month in selected_months_colorado_springs])}"

In [ ]:
print_most_similar_cade(selected_models_monthly_colorado_springs, target_word_colorado_springs)

### Temporarily changing embeddings

In [ ]:
plot_temporal_changing_embedding(target_word_colorado_springs, selected_models_monthly_colorado_springs,
                                 top_n=4, title=title_colorado_springs, subtitle=subtitle_colorado_springs, use_tsne=True)

### Cosine similarities

In [ ]:
test_words_colorado_springs = ["music", "gunman", "massacre"]

In [ ]:
plot_cosine_similarity(target_word_colorado_springs, test_words_colorado_springs, selected_models_monthly_colorado_springs, event="dec", event_name="Colorado Springs nightclub shooting")

### TPPMI

In [ ]:
tppmi_colorado_springs = tppmi_model.get_tppmi(test_words_colorado_springs + [target_word_colorado_springs])

In [91]:
target_word_colorado_springs = "colorado"
selected_months_colorado_springs = [9, 10, 11, 12]
tppmi_model.print_most_similar_words(target_word_colorado_springs, top_n = 2)

Word: colorado - Month: 09
('boeberts', 0.2607069305802309)
('wellington', 0.19660021834950225)
--------------------------------
Word: colorado - Month: 07
('calculator', 0.14771522731339093)
('belonging', 0.12593865950157634)
--------------------------------
Word: colorado - Month: 03
('rcolo', 0.2278081219982547)
('boulder', 0.14867897338377872)
--------------------------------
Word: colorado - Month: 11
('springs', 0.19245300420063183)
('boebert', 0.14048991088794388)
--------------------------------
Word: colorado - Month: 01
('coached', 0.11246103426158716)
('eileen', 0.1092298116549147)
--------------------------------
Word: colorado - Month: 04
('springs', 0.31251246204637995)
('churches', 0.19618612143587422)
--------------------------------
Word: colorado - Month: 08
('headwaters', 0.1610758166295604)
('denver', 0.1559120901869835)
--------------------------------
Word: colorado - Month: 06
('denver', 0.1670476387331983)
('intimate', 0.13148246327051127)
----------------------

In [97]:
test_words_colorado_springs = ["massacre", "kansas"]
plot_word_vectors_tppmi(tppmi_model.get_2d_representation(test_words_colorado_springs + [target_word_colorado_springs], use_tsne=False, selected_months = selected_months_colorado_springs))

All words are contained in the vocabulary
